In [0]:
import keras
from keras.utils import np_utils
print("Keras version: {}".format(keras.__version__))

Using TensorFlow backend.


Keras version: 2.2.2


# Preparation

```shell
$ pip install keras
```

You can change the backend core on file **$HOME\.keras\keras.json**. Now keras mainly supports `Tensorflow`, `CNTK`, and `Theano`.

```json
{
    "image_data_format": "channels_last",
    "floatx": "float32",
    "epsilon": 1e-07,
    "backend": "tensorflow"
}
```

# Quick Start

The core data structure of Keras is a model. The simplest type of model is `Sequential` model, a linear stack of layers. The other more complex architectures, you should use `Keras functional API` to build arbitrary graphs of layers.

## Sequential

```python
from keras.models import Sequential
from keras.layers import Dense, Activation

# ----- MODEL -----
# method.1 : model building in sequential 
model = Sequential()
model.add(Dense(output_dim=64, input_dim=100))
model.add(Activation("relu"))
model.add(Dense(output_dim=10))
model.add(Activation("softmax"))
# method.2 : the combination
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

# ----- TARGET -----
# compile network and setup loss function and optimizer
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# more complex way
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True))

# ----- LEARNING -----
# training and evaluating the model
model.fit(x_train, y_train, nb_epoch=5, batch_size=32)
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=32)

# ----- INFERENCE -----
# generates prediction
classes = model.predict(x_test, batch_size=128)
```

# Example

## Data reading

In [0]:
from keras.datasets import mnist
import numpy as np

In [0]:
# X_train, X_test: (nb_samples, 3, 32, 32) , RGB channels and data type is uint8
# Y_train, Y_test: (nb_samples,) and the value is from 0 to 9

(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Hyperparameters

In [0]:
batch_size = 128
nb_classes = 10
nb_epoch = 2

img_rows, img_cols = 28, 28
nb_filters = 32
pool_size = (2,2)
kernel_size = (3,3)

In [0]:
# (sample_num, row, col, channel)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.

# one-hot encoding
# e.g. array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

## Model

In [0]:
import os
import sys
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, MaxPooling2D, Conv2D
from keras.models import save_model, load_model

In [0]:
model = Sequential()

# first convolutional layer
model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]), \
                        padding='valid', input_shape=input_shape))
model.add(Activation('relu'))

# second convolutional layer
model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1])))
model.add(Activation('relu'))

# max pooling layer
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

# flatten
model.add(Flatten())

# First Dense layer (FC 1)
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# Second Dense Layer (FC 2)
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

## Learning

In [0]:
# define loss function, optimizer, and metrics
model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

# load the model if it exists
modelpath = os.path.join('.','keras_model','basis.h5')
if os.path.isfile(modelpath):
    model = load_model(modelpath)

## Training

In [0]:
# training the model
model.fit(x_train, y_train, \
          batch_size=batch_size, \
          epochs=nb_epoch, \
          verbose=1, \
          validation_data=(x_test, y_test), shuffle=True)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 109s 2ms/step - loss: 0.0321 - acc: 0.9898 - val_loss: 0.0256 - val_acc: 0.9924
Epoch 2/2
60000/60000 [==============================] - 122s 2ms/step - loss: 0.0299 - acc: 0.9910 - val_loss: 0.0295 - val_acc: 0.9907


## Evaluating

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Score: {}.'.format(score[0]))
print('Test Accuracy: {}.'.format(score[1]))

Test Score: 0.0295322789906284.
Test Accuracy: 0.9907.


## Prediction / Inference

In [0]:
pred = model.predict(x_test[0:2])
print(pred)
print(np.argmax(pred, axis=1))
print(y_test[0:2])

[[9.3277842e-12 2.6556640e-10 1.2631630e-09 1.1982426e-09 5.3136429e-13
  1.6533845e-12 6.8219537e-17 1.0000000e+00 4.1645437e-12 5.3488467e-09]
 [8.3918010e-08 5.3213876e-07 9.9999905e-01 5.8258606e-12 3.0362105e-11
  3.3483976e-11 3.3167058e-07 2.6970934e-12 7.3705965e-11 2.7507911e-13]]
[7 2]
[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


# Portability

## Save the model

In [0]:
save_model(model, modelpath)

## Keras HDF5 to Tensorflow PB

In [0]:
from keras import backend as K
import tensorflow as tf
print(model.output.op.name)
saver = tf.train.Saver()
saver.save(K.get_session(), os.path.join('.','keras_model','basis_model.ckpt'))

activation_4_1/Softmax


'./keras_model/basis_model.ckpt'